In [1]:
from jupyter_plotly_dash import JupyterDash

import base64
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
from bson.json_util import dumps

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/tmp/ipykernel_80413/2350576775.py:6: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

/tmp/ipykernel_80413/2350576775.py:7: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`

/tmp/ipykernel_80413/2350576775.py:9: UserWarning:


The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`



In [2]:
from modules import animal_shelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "12345678"

shelter = animal_shelter.AnimalShelter(username, password, "nv-desktop-services.apporto.com", 31943, "AAC", "animals")

In [3]:
# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))
df.head

<bound method NDFrame.head of              age_upon_outcome animal_id animal_type  \
0          4         7 months   A733653         Cat   
1          9          3 years   A720214         Dog   
2          8           1 year   A736551         Dog   
3         10         3 months   A664290         Cat   
4         11           1 year   A721199         Dog   
...      ...              ...       ...         ...   
19995   9999         2 months   A665304         Dog   
19996  10000          4 weeks   A728994       Other   
19997   9994          2 years   A713640         Cat   
19998   9995          3 years   A733093         Dog   
19999   9996          3 weeks   A678494         Cat   

                                          breed         color date_of_birth  \
0                                   Siamese Mix    Seal Point    2016-01-25   
1                        Labrador Retriever Mix     Red/White    2013-02-04   
2      Labrador Retriever/Australian Cattle Dog         Black    2015-10

In [4]:
print(len(df.to_dict(orient='records')))
print(df.columns)

20000
Index(['', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color',
       'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype',
       'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long',
       'age_upon_outcome_in_weeks'],
      dtype='object')


In [5]:
#########################
# Dashboard Layout / View
#########################

# Creating a Dash application instance
app = dash.Dash('SimpleExample')

# Logo for Grazioso Salvare
image_filename = 'Grazioso-Salvare-Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Defining the layout of the dashboard
app.layout = html.Div([
    # Displaying the customer image with Grazioso Salvare's logo
    html.Img(
        id='customer-image',
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        alt='customer image',
        style={'height':'5%', 'width':'5%'}
    ),
    
    # Displaying the title and author of the dashboard
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.H2('Rui Costa')),
    
    # Creating a horizontal line for visual separation
    html.Hr(),
    
    # Creating a radio button group for selecting the rescue type filter
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 1},
                {'label': 'Mountain Rescue', 'value': 2},
                {'label': 'Disaster Rescue', 'value': 3},
                {'label': 'All','value': 4}
            ],
            value='MTL',
            labelStyle={'display': 'inline-block'}
        ) 
    ),
    
    # Creating another horizontal line
    html.Hr(),
    
    # Creating a DataTable component for displaying data
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        fixed_columns={'headers': True, 'data': 1},
        style_table={'minWidth': '100%'},
        data=df.to_dict('records'),
        sort_action='native',
        page_size=10
    ),
    
    # Adding line break and horizontal line
    html.Br(),
    html.Hr(),
    
    # Creating a row division for displaying the chart and geolocation chart side-by-side
    html.Div(
        className='row',
        style={'display' : 'flex'},
        children=[
            # Creating a column for the chart
            html.Div(
                id='graph-id',
                className='col s12 m6',
            ),
            
            # Creating a column for the geolocation chart
            html.Div(
                id='map-id',
                className='col s12 m6',
            )
        ]
    )
])

In [6]:
#############################################
# Interaction Between Components / Controller
#############################################

# Callback function to update the dashboard based on the selected filter value
@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(value):
    # Check the selected filter value and update the dataframe accordingly
    if (value == 1):
        df = pd.DataFrame(list(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesa Bay Retr Mix", "Newfoundland Mix"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26}, "age_upon_outcome_in_weeks": {"$lte": 156}
        })))
    elif (value == 2):
        df = pd.DataFrame(list(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26}, "age_upon_outcome_in_weeks": {"$lte": 156}
        })))
    elif (value == 3):
        df = pd.DataFrame(list(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20}, "age_upon_outcome_in_weeks": {"$lte": 300}
        })))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
        
    # Define the columns for the DataTable component
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    
    # Convert the dataframe to a dictionary of records for data input to the DataTable
    data = df.to_dict('records')

    return (data, columns)


# Callback function to update the style of selected columns in the DataTable
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Callback function to update the graph based on the derived viewport data from the DataTable
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_graphs(viewData):
    if viewData is None:
        return
    
    # Convert the derived viewport data to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(
            figure=px.pie(dff, names='breed')
        )
    ]


# Callback function to update the map based on the derived viewport data from the DataTable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_map(view


In [ ]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:15751/

 * Serving Flask app "SimpleExample" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
